In [1]:
import os
from os.path import exists
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)

Mounted at /content/gdrive


In [3]:
!pip install transformers
!pip install emoji
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
# Import libraries
import torch
from transformers import AutoModel, AutoTokenizer, pipeline, AutoModelForSequenceClassification
#from TweetNormalizer import normalizeTweet
import pandas as pd
#import re
# from nltk import word_tokenize
# from nltk.corpus import stopwords
# from nltk.stem import WordNetLemmatizer
import numpy as np
#from pysentimiento_master.pysentimiento.preprocessing import preprocess_tweet
#from pysentimiento_master.pysentimiento.preprocessing import *
from tqdm import tqdm
from scipy.special import softmax
import urllib.request
import csv

In [6]:
# Load in model

model = AutoModelForSequenceClassification.from_pretrained("bhadresh-savani/distilbert-base-uncased-emotion")
tokenizer = AutoTokenizer.from_pretrained("bhadresh-savani/distilbert-base-uncased-emotion")
# Tryy wit "text-classification"
classifier = pipeline("text-classification", model = model, tokenizer = tokenizer, return_all_scores=True)

res = classifier("That was just unimaginably amazing i had no idea!!!")


/usr/local/lib/python3.7/dist-packages/transformers/pipelines/text_classification.py:90: UserWarning: `return_all_scores` is now deprecated, use `top_k=1` if you want similar functionnality
  "`return_all_scores` is now deprecated, use `top_k=1` if you want similar functionnality", UserWarning


In [9]:
res[0][5]

{'label': 'surprise', 'score': 0.5884455442428589}

In [24]:
# FUNCION THAT TAKES A DATAFRAME AS INPUT AND OUTPUTS IT WITH LABELS AND SCORES
def get_mood(df):
    
    model = AutoModelForSequenceClassification.from_pretrained("bhadresh-savani/distilbert-base-uncased-emotion")
    tokenizer = AutoTokenizer.from_pretrained("bhadresh-savani/distilbert-base-uncased-emotion")
 
    classifier = pipeline("text-classification", model = model, tokenizer = tokenizer, return_all_scores=True)
    
    # Take a list of the tweets and find the sentiment and score of each 
    tweets = list(df["text"])
    
    labels = []
    scores = []
    
    for sent in tqdm(tweets):
        
        try:
            results = classifier(sent)
            #Assign label to label
            #df.at[i,"label"] = results[0][0]["label"]
            #df.at[i,"score"] = results[0][0]["score"]
            
            labels.append(results[0][5]["label"])
            scores.append(results[0][5]['score'])
        # Get the exception to print out the error and for which team/time then pass to next month
        except:
        # except Exception as e:
        #     print(e)
            labels.append("DELETE")
            scores.append(100000)
            print("The tweet was: ",sent)
            pass

    # Create an empty col for label and score
    df = df.assign(label = labels)
    df = df.assign(score = scores)
    
    return df

In [25]:
# Loop through each pre-processed file 
os.chdir("/content/gdrive/My Drive/Diss/2. Analysis")
!ls
working_directory = os.getcwd()
working_directory

 _1_get_vader_scores.py
'1. VADER (working file).ipynb'
'2.1 Roberta Mood.ipynb'
'2.1 Run get_pysent_scores on all data .ipynb'
'2.2 Run get_pysent_scores monthly.ipynb'
'2.5 Pysentimiento v2.ipynb'
 _2_get_pysent_scores_GOOGLE.py
 _2_get_pysent_scores.py
'2. Pysent (working file).ipynb'
 _3_get_prediction_errors.py
'3. Get sentiment scores (working file).ipynb'
 BERTweet-master
 Data
 Merging_team_tweets.ipynb
 OLD
'prediction error (working file).ipynb'
 __pycache__
 pysentimiento_master
'READ ME.txt'
 TweetNormalizer.py


'/content/gdrive/My Drive/Diss/2. Analysis'

In [27]:
# List of teams and years to iterate through for file names
team_search_queries = {#'TEST': [2012]
                        'knicks': [2012],
                       'giants': [2012],
                      'devils': [2012],
                      'mets': [2012],
                      'yankees': [2012]
                       }

In [28]:
os.chdir("/content/gdrive/My Drive/Diss/2. Analysis")
working_directory = os.getcwd()

for team, years in team_search_queries.items():
  for year in years:

    # Print out to let me know progess
    print("Currently analysing: {} for {}".format(year, team))

    # Read in the preprocessed csv

    csv_file_name = "/Data/Preprocessed tweets/{}_{}_processed.csv".format(year, team)

    clean_data = pd.read_csv(working_directory + csv_file_name)

    surprise_data = get_mood(clean_data)

    surprise_data.to_csv(working_directory + "/Data/Surprise Roberta Data\{}_{}_aux_distilbert_surprise.csv".format(year, team))

Currently analysing: 2012 for knicks


100%|██████████| 23482/23482 [29:17<00:00, 13.36it/s]


Currently analysing: 2012 for giants


100%|██████████| 71694/71694 [1:27:01<00:00, 13.73it/s]


Currently analysing: 2012 for devils


100%|██████████| 29497/29497 [36:05<00:00, 13.62it/s]


Currently analysing: 2012 for mets


100%|██████████| 67212/67212 [1:20:59<00:00, 13.83it/s]


Currently analysing: 2012 for yankees


100%|██████████| 70542/70542 [1:22:03<00:00, 14.33it/s]
